### 날짜별, 기업별 기업 규모 열

In [1]:
import pandas as pd

In [20]:
import pandas as pd

# 파일 읽기
df = pd.read_csv('AR_CAR+라벨.csv', parse_dates=['사건 날짜'])

# 소속사(기업명)별 날짜 범위 확인
result = df.groupby('소속사')['사건 날짜'].agg(['min', 'max']).reset_index()

# 결과 출력
result

,소속사,min,max
0,HYBE,2021-03-09,2024-09-07
1,JYP,2016-02-21,2025-03-02
2,RBW,2023-10-31,2023-10-31
3,SM,2014-12-09,2025-04-16
4,SM C&C,2016-05-23,2023-08-15
5,YG,2014-10-14,2024-08-19
6,아티스트컴퍼니,2023-11-27,2024-11-24
7,큐브,2017-04-22,2023-08-21
8,키이스트,2014-08-18,2025-03-11


시총 데이터 하나로 묶기

In [18]:
import pandas as pd
import glob
import os

# 1. CSV 파일들이 있는 폴더 경로
folder_path = 'C:/Users/chica/Downloads'  # 실제 경로로 변경하세요
file_list = glob.glob(os.path.join(folder_path, '*.csv'))

# 2. 회사명별 DataFrame 저장용 딕셔너리
company_data = {}

# 3. 파일 하나씩 처리
for file in file_list:
    filename = os.path.basename(file)
    company_name = filename.split('_')[0]  # 언더바 이전 텍스트 추출

    # CSV 읽기 (인코딩 처리)
    try:
        df = pd.read_csv(file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='cp949')

    # 딕셔너리에 누적
    if company_name in company_data:
        company_data[company_name].append(df)
    else:
        company_data[company_name] = [df]

# 4. 회사별로 병합해서 저장
for name, df_list in company_data.items():
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(f'{name}_시가총액.csv', index=False, encoding='utf-8-sig')
    print(f'{name}_시가총액.csv 저장 완료')


HYBE_시가총액.csv 저장 완료
JYP_시가총액.csv 저장 완료
RBW_시가총액.csv 저장 완료
SM C&C_시가총액.csv 저장 완료
SM_시가총액.csv 저장 완료
YG_시가총액.csv 저장 완료
아티스트컴퍼니_시가총액.csv 저장 완료
큐브_시가총액.csv 저장 완료
키이스트_시가총액.csv 저장 완료


In [22]:
hybe = pd.read_csv("HYBE_시가총액.csv", parse_dates=['일자'])
hybe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      876 non-null    datetime64[ns]
 1   종가      876 non-null    int64         
 2   대비      876 non-null    int64         
 3   등락률     876 non-null    float64       
 4   시가      876 non-null    int64         
 5   고가      876 non-null    int64         
 6   저가      876 non-null    int64         
 7   거래량     876 non-null    int64         
 8   거래대금    876 non-null    int64         
 9   시가총액    876 non-null    int64         
 10  상장주식수   876 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(9)
memory usage: 75.4 KB


In [23]:
import pandas as pd
import glob
import os

# 현재 디렉토리에서 *_시가총액.csv 파일 모두 찾기
file_list = glob.glob('*_시가총액.csv')

for file in file_list:
    try:
        # CSV 파일 읽기 (인코딩 자동 처리)
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='cp949')

        # 중복 행 제거
        df = df.drop_duplicates()

        # '일자' 열이 존재할 경우에만 정렬
        if '일자' in df.columns:
            # 일자 열이 문자열이면 datetime으로 변환
            df['일자'] = pd.to_datetime(df['일자'], errors='coerce')
            df = df.sort_values('일자')

        # 다시 같은 파일 이름으로 저장
        df.to_csv(file, index=False, encoding='utf-8-sig')
        print(f'{file} 처리 완료')

    except Exception as e:
        print(f'{file} 처리 중 오류 발생: {e}')

HYBE_시가총액.csv 처리 완료
JYP_시가총액.csv 처리 완료
RBW_시가총액.csv 처리 완료
SM C&C_시가총액.csv 처리 완료
SM_시가총액.csv 처리 완료
YG_시가총액.csv 처리 완료
아티스트컴퍼니_시가총액.csv 처리 완료
큐브_시가총액.csv 처리 완료
키이스트_시가총액.csv 처리 완료


In [3]:
def map_closet_date(scandal_date, stock_date):
    return stock_date[stock_date>=scandal_date].min()

In [21]:
scandal = pd.read_csv("AR_CAR+라벨.csv", parse_dates=['사건 날짜'])

In [22]:
import pandas as pd
from tqdm import tqdm

# 데이터 불러오기
df = pd.read_csv("AR_CAR+라벨.csv", parse_dates=['사건 날짜(매핑된 주가 날짜)'])

# 시가총액을 저장할 리스트
market_caps = []

# 각 행마다 소속사와 날짜 기준으로 시가총액 매핑
for i in tqdm(range(len(df))):
    firm = df.loc[i, '소속사']
    date = df.loc[i, '사건 날짜(매핑된 주가 날짜)']
    
    try:
        # 시가총액 파일 경로 (파일명은 소속사명 기반)
        file_path = f"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가/{firm}_시가총액.csv"  # 실제 경로로 수정 필요
        cap_df = pd.read_csv(file_path, parse_dates=['일자'])

        # 해당 날짜 행 찾기
        row = cap_df.loc[cap_df['일자'] == date]

        if not row.empty:
            market_caps.append(row.iloc[0]['시가총액'])
        else:
            market_caps.append(None)

    except FileNotFoundError:
        print(f"{file_path} 파일이 존재하지 않습니다.")
        market_caps.append(None)

# 결과 컬럼 추가
df['시가총액'] = market_caps

  0%|          | 0/326 [00:00<?, ?it/s]

100%|██████████| 326/326 [00:03<00:00, 85.77it/s]


In [23]:
len(df)

326

In [25]:
sum(df['시가총액'].isna())

2

In [26]:
df.to_csv("AR_CAR+라벨+시가총액.csv", encoding='utf-8-sig')

결측치 채운 후

In [28]:
import pandas as pd
df = pd.read_csv("AR_CAR+라벨+시가총액.csv")
sum(df['시가총액'].isna())

0

시가 총액에 대한 로그 변환열과 랭크처리열 추가

In [29]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df = pd.read_csv("AR_CAR+라벨+시가총액+관심도.csv")

# 로그 시가총액 추가 (0 이하 시가총액이 있을 경우 NaN 처리)
df['log_시가총액'] = np.log(df['시가총액'].replace(0, np.nan))

# 분위수 랭크 (0~1 사이 값)
df['시가총액_랭크'] = df['시가총액'].rank(pct=True)

df.to_csv("AR_CAR+라벨+시가총액+관심도.csv")